<a href="https://colab.research.google.com/github/kolya-gazshev/analysis-of-A-B-testing-results/blob/main/analysis_of_A_B_testing_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**analysis of A/B testing results**

In [ ]:
# @title SQL - запит
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")
query = """with session_info as (
 select
 s.date,
 s.ga_session_id,
 sp.country,
 sp.device,
 sp.continent,
 sp.channel,
 ab.test,
 ab.test_group
from `DA.ab_test` ab
join `DA.session` s
on ab.ga_session_id = s.ga_session_id
join `DA.session_params` sp
on sp.ga_session_id = ab.ga_session_id
),
session_with_orders as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct o.ga_session_id) as session_with_orders
from `DA.order` o
join session_info
on o.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group
),
events as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 sp.event_name,
 count(sp.ga_session_id) as event_cnt
from `DA.event_params` sp
join session_info
on sp.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 sp.event_name
),
session as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct session_info.ga_session_id) as session_cnt
from session_info
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group
),
account as (
select
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group,
 count(distinct acs.ga_session_id) as new_account_cnt
from `DA.account_session` acs
join session_info
on acs.ga_session_id = session_info.ga_session_id
group by
 session_info.date,
 session_info.country,
 session_info.device,
 session_info.continent,
 session_info.channel,
 session_info.test,
 session_info.test_group)




select
 session_with_orders.date,
 session_with_orders.country,
 session_with_orders.device,
 session_with_orders.continent,
 session_with_orders.channel,
 session_with_orders.test,
 session_with_orders.test_group,
 'session with orders' as event_name,
 session_with_orders.session_with_orders as value
from session_with_orders
union all
select
 events.date,
 events.country,
 events.device,
 events.continent,
 events.channel,
 events.test,
 events.test_group,
 event_name,
 event_cnt as value
from events
union all
select
 session.date,
 session.country,
 session.device,
 session.continent,
 session.channel,
 session.test,
 session.test_group,
 'session' as event_name,
 session_cnt as value
from session
union all
select
 account.date,
 account.country,
 account.device,
 account.continent,
 account.channel,
 account.test,
 account.test_group,
 'new account' as event_name,
 new_account_cnt as value
from account"""

query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()

df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Ecuador,mobile,Americas,Organic Search,2,2,session with orders,1
1,2020-11-02,Kuwait,desktop,Asia,Organic Search,2,2,session with orders,1
2,2020-11-06,Cyprus,desktop,Asia,Direct,2,1,session with orders,1
3,2020-11-06,Ecuador,desktop,Americas,Paid Search,2,1,session with orders,1
4,2020-11-07,Tunisia,desktop,Africa,Direct,2,1,session with orders,1


In [ ]:
# @title Таблиця конверсії
import statsmodels.api as sm
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']
results = []
for i in range(1, 5):
    dfi = df[df['test'] == i]
    pivot_table = pd.pivot_table(dfi, values='value', index='event_name', columns='test_group', aggfunc="sum")
    for m in metrics:
      numerator_convertion_a = pivot_table.at[m, 1]
      domenator_convertion_a = pivot_table.at['session', 1]
      numerator_convertion_b = pivot_table.at[m, 2]
      domenator_convertion_b = pivot_table.at['session', 2]
      conversion_rate_a = numerator_convertion_a/domenator_convertion_a
      conversion_rate_b = numerator_convertion_b/domenator_convertion_b
      z_stat, p_value = sm.stats.proportions_ztest([numerator_convertion_a, numerator_convertion_b], [domenator_convertion_a, domenator_convertion_b])
      results.append({
                                    'test_number': i,
                                    'numerator_event': f"{m}/session",
                                    'numerator_event': m,
                                    'denominator_event': 'session',
                                    'numerator_converse_A': numerator_convertion_a,
                                    'denominator_converse_A' : domenator_convertion_a,
                                    'conversion_rate_A': conversion_rate_a,
                                    'numerator_converse_B': numerator_convertion_b,
                                    'denominator_converse_B': domenator_convertion_b,
                                    'conversion_rate_B': conversion_rate_b,
                                    'metric_change': ((conversion_rate_b/conversion_rate_a)-1)*100,
                                    'z_stat':z_stat,
                                    'p_value':p_value,
                                    'significant': p_value < 0.05
                                    })
df_final = pd.DataFrame(results)

df_final

,test_number,numerator_event,denominator_event,numerator_converse_A,denominator_converse_A,conversion_rate_A,numerator_converse_B,denominator_converse_B,conversion_rate_B,metric_change,z_stat,p_value,significant
0,1,add_payment_info,session,1988,45362,0.043825,2229,45193,0.049322,12.542021,-3.924884,0.000087,True
1,1,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True
2,1,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
3,1,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
4,2,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
5,2,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
6,2,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False
7,2,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,False
8,3,add_payment_info,session,3623,70047,0.051722,3697,70439,0.052485,1.474630,-0.643172,0.520112,False
9,3,add_shipping_info,session,5298,70047,0.075635,5188,70439,0.073652,-2.621211,1.413727,0.157442,False


Посилання на дашборд

https://public.tableau.com/app/profile/gazshev.nick/viz/Book1_17508668722470/AB-test?publish=yes